# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| product_id | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

- As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

In [4]:
-- impute_missing_values
WITH missing AS (SELECT quantity,
       product_id,
       discount,
       market,
       region,
       sales
FROM orders
WHERE quantity is NULL),
unit_prices AS (
SELECT o.product_id,
       CAST(o.sales/o.quantity AS NUMERIC) AS unit_price
FROM orders o
RIGHT JOIN missing AS m
	  ON o.product_id = m.product_id
	  AND o.discount = m.discount
WHERE o.quantity is NOT NULL)
SELECT DISTINCT m.*, ROUND(CAST(m.sales AS NUMERIC) / up.unit_price,0) AS calculated_quantity
FROM missing AS m
INNER JOIN unit_prices as up
      ON m.product_id = up.product_id;

SELECT order_id, order_date, COUNT(*) AS count
FROM orders
GROUP BY order_id, order_date
HAVING COUNT(*) > 1;

-- Yearly sales
SELECT 
    EXTRACT(YEAR FROM order_date::date) AS year,
	ROUND(CAST(SUM(sales) AS NUMERIC),2) AS total_sales
FROM orders
GROUP BY EXTRACT(YEAR FROM order_date::date)
ORDER BY year ASC;
-- The sales show an increasing trend...good news!

-- Most profitable products - top 5
SELECT * FROM ( 
SELECT products.product_name, 
	   COUNT(quantity) as Qty_sold,
	  ROUND(SUM(CAST(ord.profit AS NUMERIC)),2) as product_total_profit
      FROM orders AS ord
      INNER JOIN products
              ON ord.product_id = products.product_id
      GROUP BY products.product_name
      ORDER BY product_total_profit DESC
	  LIMIT 5
    ) AS sub;

--Calculate Average transaction value and profit per order
SELECT EXTRACT(YEAR FROM order_date::date) AS year,
       SUM(profit) / SUM(quantity) as profit_per_order,
       SUM(sales) / SUM(quantity) as Average_Trnx_order
FROM orders
GROUP BY year
ORDER BY year ASC
-- AVG Tranx value is around 71$ and profit per order is around 8$, remains consistent every year!



,year,profit_per_order,average_trnx_order
0,2011,7.919476,71.879204
1,2012,8.066314,70.253698
2,2013,8.454216,70.755525
3,2014,8.317100,70.933813


In [9]:
-- top_five_products_each_category
SELECT * FROM ( 
SELECT products.category,
	   products.product_name, 
	  ROUND(SUM(CAST(ord.sales AS NUMERIC)),2) as product_total_sales,
	  ROUND(SUM(CAST(ord.profit AS NUMERIC)),2) as product_total_profit,
      RANK() OVER(PARTITION BY products.category ORDER BY SUM(ord.sales) DESC) AS product_rank
      FROM orders AS ord
      INNER JOIN products
              ON ord.product_id = products.product_id
      GROUP BY products.category, products.product_name
	) AS tmp
      WHERE product_rank < 6;    

,category,product_name,product_total_sales,product_total_profit,product_rank
0,Furniture,"Hon Executive Leather Armchair, Adjustable",58193.48,5997.25,1
1,Furniture,"Office Star Executive Leather Armchair, Adjust...",51449.80,4925.80,2
2,Furniture,"Harbour Creations Executive Leather Armchair, ...",50121.52,10427.33,3
3,Furniture,"SAFCO Executive Leather Armchair, Black",41923.53,7154.28,4
4,Furniture,"Novimex Executive Leather Armchair, Adjustable",40585.13,5562.35,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.60,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5
